In [1]:
from torch.optim import Adam, SGD
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from inverse_modelling_tfo.models import train_model, create_perceptron_model, train_model_wtih_reporting
from inverse_modelling_tfo.data import generate_data_loaders, equidistance_detector_normalization, constant_detector_count_normalization
from inverse_modelling_tfo.data.intensity_interpolation import get_interpolate_fit_params_custom, interpolate_exp
from inverse_modelling_tfo.data.interpolation_function_zoo import *
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import os

# Set my GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

data = pd.read_pickle(
    r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/s_based_intensity.pkl')
equidistance_detector_normalization(data)

# Interpolate intensity to remove noise
data = interpolate_exp(data, weights=[1, 1])
data['Intensity'] = data['Interpolated Intensity']

# Far values wayy to small to affect anything. Take log
data['Intensity'] = np.log10(data['Intensity'])
data.head()

data = pd.pivot(data, index=['Maternal Wall Thickness', "Maternal Hb Concentration", "Maternal Saturation",
                "Fetal Hb Concentration", "Fetal Saturation"], columns=["SDD", "Wave Int"], values="Intensity").reset_index()
# Slight coding mistake, not all waveints have both wv1 and 2
data.dropna(inplace=True)
data.head()

# Rename multi-index columns
data.columns = ['_'.join([str(col[0]), str(col[1])])
                if col[1] != '' else col[0] for col in data.columns]
# y_columns = ['Maternal Wall Thickness', "Maternal Hb Concentration", "Maternal Saturation", "Fetal Hb Concentration", "Fetal Saturation"]
y_columns = ['Fetal Saturation']
x_columns = list(filter(lambda X: '_' in X, data.columns))

# filtered_fitting_param_table = fitting_param_table[fitting_param_table['Wave Int'] == 2.0]
# x_scaler = preprocessing.StandardScaler()
y_scaler = preprocessing.StandardScaler()
data[y_columns] = y_scaler.fit_transform(data[y_columns])

# Manual log(intensity) normalization
# stddev.   (Actual value is higher but let's keep it here for now)
data[x_columns] /= 100.0
data[x_columns] += 0.5  # unit var : mean


In [2]:
np.random.seed(70)  # Set seed for consistentcy
params = {
    'batch_size': 32, 'shuffle': True, 'num_workers': 2
}
train, val = generate_data_loaders(data, params, x_columns, y_columns, 0.8)


In [3]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv1d(20, 4, 5)
        self.conv2 = nn.Conv2d(20, 4, 5)
        self.fc1 = nn.Linear(10, 2)  # 5*5 from image dimension
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x1 = F.relu(self.conv1(x[:, :20]))
        x2 = F.relu(self.conv2(x[:, 20:]))

        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flatten all dimensions except the batch dimension
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()


In [4]:
for i in train:
    a = i
    break


In [5]:
a = a[0]


In [11]:
a = a.view(-1, 2, 20)
a.shape


torch.Size([32, 2, 20])

In [12]:
conv = nn.Conv1d(2, 4, 5, groups=2)
x = F.relu(conv(a))
x.shape

torch.Size([32, 4, 16])

In [10]:
x = nn.Flatten()(x)
x.shape

torch.Size([32, 64])

In [13]:
class TwoChannelCNN(nn.Module):
    """A 2 channel based CNN connected to a set of FC layers. The 2 input channels of the CNN each 
    take half of the inputs. (Say if [input_length] is 40, one channel would get the first 20 and 
    the next 20 would be fed to another channel). The CNN Outputs onto [cnn_out_channel] number of 
    output channels. (NOTE: cnn_out_channel has to be divisible by 2. The first half of out channle 
    only  see the first half of the inputs and the second half see the second half of the inputs)
    
    Afterwards, they are connected to a set of linear layers with activations. The output length for
    each of these linear layers are supploed using [linear_array]
    """
    def __init__(self, input_length, cnn_out_channel, kernel_size, linear_array) -> None:
        assert cnn_out_channel % 2 == 0, "cnn_out_channel has to be divisible by 2"
        super().__init__()
        self.split_point = input_length//2
        self.conv1 = nn.Conv1d(2, cnn_out_channel, kernel_size, groups=2)
        self.conv_output_length = cnn_out_channel * (self.split_point + 1 - kernel_size)
        self.linear_layers = [nn.Linear(self.conv_output_length, linear_array[0])]
        for index, count in enumerate(linear_array[0:-1]):
            self.linear_layers.append(nn.ReLU())
            self.linear_layers.append(nn.Linear(count, linear_array[index + 1]))
        self.linear_layers.append(nn.Flatten())
        self.linear_network = nn.Sequential(*self.linear_layers)
    
    def forward(self, x):
        x = x.view(-1, 2, self.split_point)
        x = F.relu(self.conv1(x))
        x = nn.Flatten()(x)
        x = self.linear_network(x)
        return x

In [14]:
model = TwoChannelCNN(40, 4, 5, [4, 1])

In [15]:
x = model(a)

In [17]:
x.shape

torch.Size([32, 1])